In [ ]:
import torch
from peft import LoraConfig, get_peft_model
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, BitsAndBytesConfig, AutoModelForSeq2SeqLM
from datasets import load_from_disk

from utils import print_trainable_parameters, prepare_prompt,  evaluate_model
import transformers

import json
import time

r=128
model_name = "t5-base" # "mt5-base"

/users/eleves-b/2022/ayoub.melliti/LLM project/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset

In [ ]:
# Load dataset
dataset_split = load_from_disk('dataset_split')

print(dataset_split)

cache_dir = "/Data/gabriel-mercier/slm_models"



## Fine Tune

In [ ]:
# Load tokenizer
if model_name == "t5-base":
    final_name = "google-t5/t5-base"
elif model_name == "mt5-base":
    final_name = "google/mt5-base"

tokenizer = AutoTokenizer.from_pretrained(final_name, cache_dir=cache_dir)

# Configure BitsAndBytes
bnb_config = BitsAndBytesConfig(load_in_4bit=True, 
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_quant_type='nf4',
                            )

# Load model
model_raw = AutoModelForSeq2SeqLM.from_pretrained(final_name, 
                                              cache_dir=cache_dir,
                                              trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")

print(model_raw)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear4bit(in_features=768, out_features=768, bias=False)
              (k): Linear4bit(in_features=768, out_features=768, bias=False)
              (v): Linear4bit(in_features=768, out_features=768, bias=False)
              (o): Linear4bit(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear4bit(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
        

In [ ]:
# Configure LoRA
lora_config = LoraConfig(r=r, 
                            lora_alpha=2*r,
                            target_modules=["q", "k", "v", "o"],
                            lora_dropout=0.05,
                            bias='none',
                            task_type="SEQ_2_SEQ_LM")

# Apply LoRA to model
model = get_peft_model(model_raw, lora_config)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# Configure generation settings
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.do_sample = True

# Print trainable parameters
perc_param = print_trainable_parameters(model)


### Example


In [8]:
assistant_start = "Résumé concis et structuré (100 mots maximum) :"
summary_data = dataset_split['train'][1]['summary']
prompt = prepare_prompt(dataset_split['train'][1], summary_included=False)
print('=== PROMPT ===')
print(prompt)

encoding = tokenizer(prompt, return_tensors="pt").to(device)

with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

start_index = prediction.find(assistant_start)
if start_index != -1:
    response_start = start_index + len(assistant_start)
else:
    response_start = -1 

print("=== GENERATED SUMMARY ===")
print(prediction[response_start+1:])
print(len(prediction[response_start+1:].split()))

print("=== LABEL SUMMARY ===")
print(summary_data)
print(len(summary_data.split()))

=== PROMPT ===
Résume précisément le texte suivant en français en 100 mots maximum. Concentre-toi sur les points essentiels sans ajouter d'opinions ni de commentaires. Évite les phrases inutiles et reformule les idées clairement.

Texte :
Le 24 août 1991, des Moscovites accompagnent les dépouilles des trois victimes du putsch, tuées trois jours plus tôt. Gueorgui Pinkhassov/Magnum C’était encore le temps des vacances, du repos dans les datchas. Le major-colonel du KGB Valeri Chiriaïev dormait dans la sienne, une bicoque sans eau, quelque part dans un trou perdu de la région de Iaroslav, quand la « boîte » fixée au mur s’est mise à grésiller. « On ne savait même plus si ce machin marchait encore », raconte-t-il. L’antique haut-parleur avait été installé dans toutes les demeures soviétiques pour prévenir d’une guerre nucléaire. Le message contenait aussi un code secret pour toutes les forces de sécurité. Et ce matin du 19 août 1991, il y a vingt-cinq ans, Valeri Chiriaïev découvre, stupé

In [ ]:
# Preprocess function
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], padding="max_length", max_length=2500, truncation=True)
    labels = tokenizer(examples["summary"], padding="max_length", max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:

# Preprocess datasets
dataset_train = dataset_split["train"].map(preprocess_function)
dataset_val = dataset_split["validation"].map(preprocess_function)

# Remove unnecessary columns
dataset_train = dataset_train.remove_columns(["text", "summary"])
dataset_val = dataset_val.remove_columns(["text", "summary"])

print(dataset_train)
print(dataset_val)

In [11]:

# Create a data collator for seq2seq tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
# Training arguments
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=3,
    logging_steps=1,
    evaluation_strategy="epoch",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
)

# Trainer setup
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
)


In [ ]:
# Train model
start_time = time.time()
trainer.train()
end_training = time.time()


Epoch,Training Loss,Validation Loss
1,3.697000,2.618326
2,2.987600,2.571890
3,3.523800,2.567580


TrainOutput(global_step=375, training_loss=3.498820992787679, metrics={'train_runtime': 337.177, 'train_samples_per_second': 4.449, 'train_steps_per_second': 1.112, 'total_flos': 453719162880000.0, 'train_loss': 3.498820992787679, 'epoch': 3.0})

In [ ]:
# Save model
trainer.save_model(f"./t5_r{r}")

# Log training information
logs = trainer.state.log_history
train_losses = [log["loss"] for log in logs if "loss" in log]
eval_losses = [log["eval_loss"] for log in logs if "eval_loss" in log]

with open(f"./t5_r{r}_infos.json", "w") as f:
    json.dump({"train_losses": train_losses, "eval_losses": eval_losses, "perc_training":perc_param, "time_training":end_training-start_time}, f)



## Evaluation

### Example

In [14]:
assistant_start = "Résumé concis et structuré (100 mots maximum) :"
summary_data = dataset_split['train'][1]['summary']
prompt = prepare_prompt(dataset_split['train'][1], summary_included=False)
print('=== PROMPT ===')
print(prompt)

encoding = tokenizer(prompt, return_tensors="pt").to(device)

with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

start_index = prediction.find(assistant_start)
if start_index != -1:
    response_start = start_index + len(assistant_start)
else:
    response_start = -1 

print("=== GENERATED SUMMARY ===")
print(prediction[response_start+1:])
print(len(prediction[response_start+1:].split()))

print("=== LABEL SUMMARY ===")
print(summary_data)
print(len(summary_data.split()))

=== PROMPT ===
Résume précisément le texte suivant en français en 100 mots maximum. Concentre-toi sur les points essentiels sans ajouter d'opinions ni de commentaires. Évite les phrases inutiles et reformule les idées clairement.

Texte :
Le 24 août 1991, des Moscovites accompagnent les dépouilles des trois victimes du putsch, tuées trois jours plus tôt. Gueorgui Pinkhassov/Magnum C’était encore le temps des vacances, du repos dans les datchas. Le major-colonel du KGB Valeri Chiriaïev dormait dans la sienne, une bicoque sans eau, quelque part dans un trou perdu de la région de Iaroslav, quand la « boîte » fixée au mur s’est mise à grésiller. « On ne savait même plus si ce machin marchait encore », raconte-t-il. L’antique haut-parleur avait été installé dans toutes les demeures soviétiques pour prévenir d’une guerre nucléaire. Le message contenait aussi un code secret pour toutes les forces de sécurité. Et ce matin du 19 août 1991, il y a vingt-cinq ans, Valeri Chiriaïev découvre, stupé

### Evaluation

In [ ]:
# Load fine-tuned model
model = AutoModelForSeq2SeqLM.from_pretrained(f"./t5_r{r}")
model.to(device)

In [15]:
dataset_test = dataset_split['test']

In [ ]:
# Evaluate fine-tuned model
rouges_results_finetune, bert_results_finetune = evaluate_model(model, dataset_test, tokenizer, device, generation_config)

100%|██████████| 10/10 [00:23<00:00,  2.40s/it]


BERTScore - Precision: 0.5700, Recall: 0.4051, F1: 0.4732
ROUGEScores - {'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'rougeLsum': np.float64(0.0)}




In [ ]:
results_finetune = {
    "rouge": rouges_results_finetune,
    "bert": bert_results_finetune
}

with open(f"t5_evaluation_results_finetune_r{r}.json", "w") as f:
    json.dump(results_finetune, f, indent=4)


In [ ]:

# Evaluate raw model
rouges_results_raw, bert_results_raw = evaluate_model(model_raw, dataset_test, tokenizer, device, generation_config)


results_raw = {
    "rouge": rouges_results_raw,
    "bert": bert_results_raw
}

with open("t5_evaluation_results_raw.json", "w") as f:
    json.dump(results_raw, f, indent=4)


100%|██████████| 10/10 [00:20<00:00,  2.04s/it]

BERTScore - Precision: 0.6403, Recall: 0.4503, F1: 0.5285
ROUGEScores - {'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'rougeLsum': np.float64(0.0)}


